<a href="https://colab.research.google.com/github/gautamvdblt/vanderbilt-ml/blob/main/FIFA-WorldCup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Machine Learning Using Fifa World Cup Data**

Fifa world cup notebook

## Background

The FIFA World Cup is the most prestigious football tournament in the world. The championship has been awarded every four years since the start of the tournament in 1930.

In this Project, we will use https://www.kaggle.com/datasets/brenda89/fifa-world-cup-2022.


## Project Description

The FIFA World Cup involves a qualification phase, which takes place over the preceding three years, to determine which teams quality for the tournament. In the tournament, 32 teams, including the host nation, compete for the title at different stadiums in the host country.

This dataset provides a complete overview of all international soccer matches played since the 90s. On top of that, the strength of each team is provided by incorporating actual FIFA rankings as well as player strengths based on the EA Sport FIFA video game.

## Performance Metric

In this project we will try to 
- predict what team is most likely to win the 2022 FIFA World Cup?
· What team has the strongest defense, midfield, and offense players?
· What team has the longest winning streak?


## Load Libraries

First, we'll load our standard libraries for loading data, basic exploratory data analsysi (EDA), and machine learning. 

In [7]:
#tables and visualizations
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#machine learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler
from sklearn import config_context
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

#machine learning


## Load Data

Here we first load the data into python using pandas and read it in as a pandas dataframe which is the format which we will use throughout the example. 

In [8]:
penguins = pd.read_csv('https://github.com/gautamvdblt/vanderbilt-ml/blob/main/international_matches.csv')
display(penguins.head())
penguins.info()

ParserError: ignored

In [ ]:
penguins.isna().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
year                  0
dtype: int64